# imports

In [ ]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics.SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist, euclidean


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *
from PredictionPipeline.PredictionFunctions import *
from TCR_feature_generation.SubsamplingFunctions import *

import os
from Utils import cacheOnDisk
from SegalQueue.qp import qp,fakeqp
from addloglevels import sethandlers

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression
import shap
from sklearn import metrics, preprocessing
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,SelectKBest,chi2,mutual_info_classif,f_classif
from sklearn.naive_bayes import GaussianNB

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

# plan

Function 1:
2. subsample PNP and cardio, compare phenotypes
3. produce common TCRdfs to use with permanova and mantel
4. run permanova/mantel pipeline to define which phenotypes are in interaction with TCRdf structure
Function 2: 
5. generate matched cohorts based on phenotypes described in 4
6. compare phenotypes to validate

function 3: when the matched cohorts are good:
7. generate a folder with all samples
8. compare TCR features
9. generate TCRdf
10. compare sharing rates between PNP and cardio
11. try to seperate by PCA on 10perc binary TCR
12. find unique sequences per cohort and their identity

# Function 1: subsampling, generate TCRdfs and run permanova/mantel

## define function:

function testPhenotypeAffectsOnsubsampledCohorts was copied to subsamplingFunction.py

## run for ss=12500

In [ ]:
ss=12500
repeat=1
ssPNP=False #True/False. subsample only if not exist
ssCardio=False #True/False. subsample only if not exist
genTCRdfPNP=False #True/False. 
genTCRdfCardio=False #True/False. 


testPhenotypeAffectsOnsubsampledCohorts(ss,repeat,ssPNP,ssCardio,genTCRdfPNP,genTCRdfCardio)

# function 2: generate matched cohorts and compare phenotypes

## improve matching function:

function findClosestSample was copied to subsamplingFunction.py

## generate well-matched cohorts:

### prepare phenotype DF:

In [ ]:
# get full phenotype DFs for PNP and Cardio:
f1='%s/TCR_real_data/NewPhenotypicData/PNP530_phen_new_dummies.xlsx' %MyPath
PNP530_phen_new_dummies=pd.read_excel(f1).set_index('BD')

f2='%s/TCR_real_data/CardioSamples/phenotypicData/Cardio126_phen_new_dummies.xlsx' %MyPath
Cardio126_phen_new_dummies=pd.read_excel(f2).set_index('BD')

#filter for ss12500 samples and save:
folder='%s/TCR_real_data/PNP530_SubSampled12500data_rep1/SamplesForAnalysis_corrected' %MyPath
PNP530ss12500sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)

PNP530ss12500_phen_new_dummies=PNP530_phen_new_dummies.loc[PNP530ss12500sampleList,:]

f3='%s/TCR_real_data/NewPhenotypicData/PNP530ss12500_phen_new_dummies.xlsx' %MyPath
PNP530ss12500_phen_new_dummies.to_excel(f3)


folder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled12500data_rep1/SamplesForAnalysis_corrected' %MyPath
Cardio126ss12500sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)

Cardio126ss12500_phen_new_dummies=Cardio126_phen_new_dummies.loc[Cardio126ss12500sampleList,:]

f4='%s/TCR_real_data/CardioSamples/phenotypicData/Cardio126ss12500_phen_new_dummies.xlsx' %MyPath
Cardio126ss12500_phen_new_dummies.to_excel(f4)


### run matching function:

In [ ]:
f3='%s/TCR_real_data/NewPhenotypicData/PNP530ss12500_phen_new_dummies.xlsx' %MyPath
PNP530ss12500_phen_new_dummies=pd.read_excel(f3).set_index('BD')

f4='%s/TCR_real_data/CardioSamples/phenotypicData/Cardio126ss12500_phen_new_dummies.xlsx' %MyPath
Cardio126ss12500_phen_new_dummies=pd.read_excel(f4).set_index('BD')

phenotypeList=[('Gender_Male','identity',None,True),('Age','delta',9,True),('Smoking_Past','identity',None,True),('Smoking_Yes','identity',None,True),
               ('BMI','delta',4,False),('eGFR_CKD-EPI_new','delta',None,False),('WBC','delta',3,False)]
columnsToUse=[x[0] for x in phenotypeList]
restrictedPhenotypes=[x[0] for x in phenotypeList if x[3]==True]
print restrictedPhenotypes

cohort1phenotypeDF=Cardio126ss12500_phen_new_dummies
cohort1name='Cardio126ss12500'
cohort2phenotypeDF=PNP530ss12500_phen_new_dummies
cohort2name='PNP530ss12500'

folderToSaveLists='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss12500rep1' %MyPath
random_state=5
               
matchedCardio12500samples,matchedPNP12500samples=gen_matched_cohorts_by_phenotypes(cohort1phenotypeDF,cohort1name,cohort2phenotypeDF,cohort2name, 
                                      phenotypeList, columnsToUse,restrictedPhenotypes,folderToSaveLists,random_state)
                
#phenotypeList is a list of tuples, in each tuple the first item is the phenotype name (string)the second is the conditionType-'identity' or 'delta',
#the third is the delta (int) or None and the forth is isStopingCrit (True/False)

# function 3: generate a folder with all samples, compare features, seperate by PCA and find unique sequences  

## define function:

function compareTCRforMatchedCohorts was copied to subsamplingFunction.py

In [ ]:
def compareTCRforMatchedCohorts(ss,repeat,PNPmatchedSampleList,CardioMatchedSampleList,
                                copySamples,compareFeatures,compareSharing, genSepTCRdf,nSharedMin,
                                delFolderFirst=False,genOrigComTCRdf=False,genComTCRdf=True,n_comp=None,percSharedCom=10,
                                identityColumnForPie='Pathology_McPAS'):
    
    from shutil import copyfile,rmtree
    
    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss%srep%s' %(MyPath,ss,repeat)
    FeatureMeanSummary_all_dataSets=[]
    
    #(1) generate a folder with all matched samples:
    
    matchedSampleFolder='%s/SamplesForAnalysis_corrected' %datasetFolder
    if copySamples:
        if delFolderFirst:
            rmtree(matchedSampleFolder)
        if not isdir(matchedSampleFolder):
            makedirs(matchedSampleFolder)
        #look for data files of all PNP samples that were matched to cardio sample and copy them to the common folder:
        PNPsourceFolder='%s/TCR_real_data/PNP530_SubSampled%sdata_rep%s/SamplesForAnalysis_corrected' %(MyPath,ss,repeat)
        CardiosourceFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled%sdata_rep%s/SamplesForAnalysis_corrected' %(MyPath,ss,repeat)

        folderTuple=[('PNP',PNPsourceFolder,PNPmatchedSampleList),('Cardio',CardiosourceFolder,CardioMatchedSampleList)]
        for folder in folderTuple:
            print 'now copying files from %s folder' %folder[0]
            files=[f for f in listdir(folder[1]) if isfile(join(folder[1],f))]
            print 'number of %s files = %s' %(folder[0],len(files))
            count=0                           
            for f in files:
                    f1=f.replace('b','')
                    f1=f1.replace('a','')
                    f1=f1.split('_')[0]
                    f1=f1.split('.')[0]
        #             print f1
                    if f1 in folder[2]:
                        src='%s/%s' %(folder[1],f)
                        dst='%s/%s' %(matchedSampleFolder,f)
                        copyfile(src, dst)
                        count=count+1
                    else:
    #                     print 'file %s is not in the %s matched samples list' %(f,folder[0])
                        pass

            print '%s files were copied from %s folder' %(count,folder[0])
        print 'in total, %s files were copied' %len([f for f in listdir(matchedSampleFolder) if isfile(join(matchedSampleFolder,f))])
        print 'done copying sample data files'
    
    #(2) compare features:
    if compareFeatures:
        data_folder1='TCR_real_data/CardioSamples/Cardio126_SubSampled%sdata_rep%s' %(ss,repeat)
        data_folder2='TCR_real_data/PNP530_SubSampled%sdata_rep%s' %(ss,repeat)
        datasetName1='Cardio126_ss%s_rep%s' %(ss,repeat)
        datasetName2='PNP530_ss%s_rep%s' %(ss,repeat)

        TakeSameSamples=False
        filteringList1=CardioMatchedSampleList
        filteringList2=PNPmatchedSampleList
        filteringList1Name='CardioMatchedss%srep%s' %(ss,repeat)
        filteringList2Name='PNPMatchedss%srep%s' %(ss,repeat)

        print 'comparing features between datasets:'
        FeatureMeanSummary_all_dataSets=compare_features_between_datasets(data_folder1, datasetName1,  data_folder2, datasetName2,TakeSameSamples, 
                                          filteringList1,filteringList2,filteringList1Name,filteringList2Name)

        print 'comparing geneset usage between datasets:'
        plotType='bar'
        plot_gene_usage_comparison(data_folder1, datasetName1,  data_folder2, datasetName2,plotType, TakeSameSamples, 
                                   filteringList1,filteringList2,filteringList1Name,filteringList2Name)

        #check significance of the results for different types of results:
        FeatComp=FeatureMeanSummary_all_dataSets.copy()
        FeatComp_T=FeatComp[FeatComp.index.str.contains('_T')]

        ## generate the geneUsageDF:
        regexList=[('vFamily','V.._T'),('vGene','V..-.._T'),('jGene','J..-.._T'),('dFamily','D.._T'),
                  ('VJ','V.._J.._T'),('DJ','D.._J..-.._T')]
        df_list=[]
        for regex in regexList:
            regex_item= re.compile(regex[1])
            indices=[n for n in FeatComp.index if re.match(regex_item, n)]
            df=FeatComp.loc[indices, :]
            df_list.append(df)

        geneUsageResults_T=pd.concat(df_list)
        print len(geneUsageResults_T)

        nonGeneUsageResults_T=FeatComp.loc[[x for x in FeatComp_T.index if x not in geneUsageResults_T.index], :]
        print len(nonGeneUsageResults_T)

        ## check sig with FDR for each result df seperately:
        resultDFlist=[('all',FeatComp),('onlyTotal',FeatComp_T),('onlyGeneUsage_T',geneUsageResults_T),('onlyFeatures_T',
                nonGeneUsageResults_T)]
        resultFolder='%s/featureSummaryDFs/comparisonResults' %datasetFolder
        if not isdir(resultFolder):
            makedirs(resultFolder)

        for resultDF in resultDFlist:
            print '******** %s *********' %resultDF[0]
            nTests=len(resultDF[1])
            print nTests
            FDR=0.1

            pValueColumnList=['ks_p','t_p']
            for pValueColumn in pValueColumnList:
                print pValueColumn
                resultDF_FDR=add_corrected_pValues(resultDF[1],pValueColumn,nTests,FDR)
                resultDF_FDR=resultDF_FDR.rename(columns={'Sig by bonferroni corrected pVal':'%s_Sig by bonferroni corrected pVal'\
    %pValueColumn, 'sig. by FDR=0.1':'%s_sig. by FDR=0.1' %pValueColumn})
                sigResults=resultDF_FDR[resultDF_FDR['%s_sig. by FDR=0.1' %pValueColumn]==1]
                print 'number of sig results with FDR=0.1 is %s' %len(sigResults)
                print 'sig results:'
                print sigResults
                f1='%s/sigResults_%s_%s.xlsx' %(resultFolder,resultDF[0],pValueColumn)
                sigResults.to_excel(f1)
                print 'results were saved to file'
    #(3) generate TCRdf, calculate sharing rates and seperate by PCA:
    # general definition, define anyway so they will be accessible: 
    print '*****generating %s perc TCRdfs******' %percSharedCom
    
    publicAnalysisFolder='%s/publicAnalysis' %datasetFolder
    if not isdir(publicAnalysisFolder):
        makedirs(publicAnalysisFolder)
    
    DFtype='TCR'
    genDF=genOrigComTCRdf #False=generate new df
    toBinary=True
    mbLevel='g'
    useShortName=True
    

    datasetName='MatchedSamples_ss%srep%s' %(ss,repeat)
    minVal=None    #minVal can be None,0, float, or 'dfMinVal' or dfMinVal2:
    minSharedT=None #minimal number of samples shared by seq/species required to leave sample in the database (int or None)
    percShared=percSharedCom #minimal number of samples shared by seq/species required to leave sample in the database (int [ percentage]
                    #or None)
    removeOutliers=True
    normData=True
    logTransform=True
    extractUniqueAA=True # use True when this is the first time to analyze this dataset, otherwise, use False
    minNshared=2
    onlyProductive=True
    mbDataFolder='AllSeqProjects'

    SampleList=[f for f in listdir(matchedSampleFolder) if isfile(join(matchedSampleFolder,f))]
    SampleListName='MatchedSamples_ss%srep%s' %(ss,repeat)
    libPrepMethod=None
    filterGenotek=True
    filterMinimalReads=9000000
    filterlibPrepMethod=libPrepMethod
    groupFunction='noOutlierMean'
    nSTD=5
    nMinSamples=3
    ignoreNotSameYear=True
    removeSamePerson=False
    
    
    if genComTCRdf:
        print 'generating %sperc TCRdfs' %percSharedCom
        TCRdf0=genTCRorMBdfWithManipulations(DFtype,genDF,toBinary,removeOutliers,normData,logTransform,
                                         minVal,minSharedT,percShared,
                                         mbLevel,useShortName,datasetFolder,datasetName,extractUniqueAA,
                                        minNshared,onlyProductive,mbDataFolder,SampleList,
                                          SampleListName,filterlibPrepMethod,filterGenotek, 
                                          groupFunction, nSTD, nMinSamples, ignoreNotSameYear, removeSamePerson)
    
    print 'loading %sperc TCRdfs' %percSharedCom
    TCRdfFileName='sharingMatrix_MatchedSamples_ss%srep%s_minNshared2_RA_onlyProductiveTrue__percShared%s_OLtrimmed_binary' %(ss,repeat,percSharedCom)
    TCRdfShortName='TCRdf_percShared%s_binary' %percSharedCom
    f1='%s/%s_sharingAnalysis/%s' %(datasetFolder,SampleListName,TCRdfFileName)
    TCRdf=pd.read_pickle(f1)
    print 'TCRdf shape is %s_%s' %(TCRdf.shape[0],TCRdf.shape[1])
    print 'TCRdf head:'
    print TCRdf.iloc[:4,:4]
    
    ##get PCAdf:
    ## **generate PCs as features:
    if n_comp is None:
        n_comp=5
    try:
        TCRdf=TCRdf.set_index('BD')
    except:
        print 'TCRdf already has BD column as index'
    print 'generating PCAdf with %s PCs based on TCRdf:' %n_comp
    fig2,ax2=plt.subplots(figsize=(8,6))
    print 'percSharedCom=%s' %percSharedCom
    print 'percShared=%s' %percShared
    PCAdf,ax2,p_ttest_PC1,p_ttest_PC2=PCAfunc(TCRdf,n_comp,isSparse=True,ax=ax2)
    ax2.set_title('PC1 - PC2 TCRdf %sperc-shared sequences' %percShared)
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    ax2.annotate('p_ttest_PC1=%s\np_ttest_PC2=%s' %(round(p_ttest_PC1,4),round(p_ttest_PC2,4)), xy=(0.96, 0.95), xycoords='axes fraction', 
                    fontsize=12, horizontalalignment='right', verticalalignment='top', fontweight='bold')
    print 'PCAdf shape is %s_%s' %(PCAdf.shape[0],PCAdf.shape[1])
#     print 'PCAdf HEAD:'
    PCAplotFile='%s/PC1PC2_%spercShared' %(publicAnalysisFolder, percSharedCom)
    fig2.savefig(PCAplotFile,dpi=200)
#     print PCAdf.iloc[:4,:4]
    
    #(4) compare public sequences between cohorts:
    ## number of public sequence per sample distributions, total numbers of shared sequences, identity of shared sequences
    if compareSharing:
        TCRdfInfoTuple=[('PNP',PNPmatchedSampleList,'PNPmatchedSampleList'),('Cardio',CardioMatchedSampleList,'CardioMatchedSampleList')]
        TCRdfDict={}
        uniqueSeqDict={}
        if genSepTCRdf:
            #loop over PNP and Cardio matched samples (seperately) and generate TCRdf of shared sequences for each, seperately
            for TCRdfInfo in TCRdfInfoTuple:
                minSharedT=2 #minimal number of samples shared by seq/species required to leave sample in the database (int or None)
                percShared=None #minimal number of samples shared by seq/species required to leave sample in the database (int [ percentage]
                                #or None)
                removeOutliers=True
                normData=False
                logTransform=False
    #             extractUniqueAA=False

                SampleList=TCRdfInfo[1]
                SampleListName=TCRdfInfo[2]

                print 'generating TCRdf only for %s samples from the matched samples:' %TCRdfInfo[0]
                TCRdfDict['%s' %TCRdfInfo[0]]=genTCRorMBdfWithManipulations(DFtype,genDF,toBinary,removeOutliers,normData,logTransform,
                                                     minVal,minSharedT,percShared,
                                                     mbLevel,useShortName,datasetFolder,datasetName,extractUniqueAA,
                                                    minNshared,onlyProductive,mbDataFolder,SampleList,
                                                      SampleListName,filterlibPrepMethod,filterGenotek, 
                                                      groupFunction, nSTD, nMinSamples, ignoreNotSameYear, removeSamePerson)

        ## load number of unique sequence per sample:
        f3='%s/featureSummaryDFs/PNP530Cardio126_ss%srep%s_Matched_allFeatures' %(datasetFolder,ss,repeat)
        featuresDF=pd.read_pickle(f3)
        featuresDF=editSampleNames(featuresDF)
        nUnique=pd.DataFrame(featuresDF['NT count_1'])
        ##load binary DFs and compare number of shared sequences per sample distribution between PNP and cardio:
        ## +load uniqueAA file for each
        for TCRdfInfo in TCRdfInfoTuple:     
            print 'loading seperate minSharedT2 TCRdfs for %s' %TCRdfInfo[0]
            TCRdfFileName='sharingMatrix_%s_minNshared2_RA_onlyProductiveTrue__minSharedT2_OLtrimmed_binary'  %datasetName
            f2='%s/%s_sharingAnalysis/%s' %(datasetFolder,TCRdfInfo[2],TCRdfFileName)
            TCRdfDict['%s' %TCRdfInfo[0]]=pd.read_pickle(f2)
            print '%s TCRdf shape is %s_%s' %(TCRdfInfo[0],TCRdfDict['%s' %TCRdfInfo[0]].shape[0],TCRdfDict['%s' %TCRdfInfo[0]].shape[1])
            print '%s TCRdf head:' %TCRdfInfo[0]
            print TCRdfDict['%s' %TCRdfInfo[0]].iloc[:4,:4]
            
            print 'loading uniqueSeqWithCounts....'
            f4='%s/%s_sharingAnalysis/AllUniqueWithCounts' %(datasetFolder,TCRdfInfo[2])
            AllUniqueWithCounts=pd.read_pickle(f4)
            AllUniqueWithCounts=AllUniqueWithCounts[AllUniqueWithCounts['nShared']>=nSharedMin] # take only shared by more than nShared
            UniqueSequencesWithCounts=AllUniqueWithCounts.drop(['Sample','frequencyCount (%)','prod_stat','isPublic'], axis=1)
            UniqueSequencesWithCounts= UniqueSequencesWithCounts[~UniqueSequencesWithCounts.index.duplicated(keep='first')]   
            uniqueSeqDict['%s' %TCRdfInfo[0]]=UniqueSequencesWithCounts

            
        #calculate percentage of public sequences out of all unique and plot distributions of this statistic in both cohorts:
        PNP_nShared=pd.merge(pd.DataFrame(TCRdfDict['PNP'].sum(axis=1)),nUnique, how='left', left_index=True,right_index=True)
        Cardio_nShared=pd.merge(pd.DataFrame(TCRdfDict['Cardio'].sum(axis=1)),nUnique, how='left', left_index=True,right_index=True)
        PNP_nShared['percPublic']=100*PNP_nShared[0]/PNP_nShared['NT count_1']
        Cardio_nShared['percPublic']=100*Cardio_nShared[0]/Cardio_nShared['NT count_1']
         
        data1=Cardio_nShared['percPublic']
        data2=PNP_nShared['percPublic']
        data1=data1[data1.notnull()].tolist()
        data2=data2[data2.notnull()].tolist()
        data1Name='Cardio'
        data2Name='PNP'
        folderToSave=publicAnalysisFolder
        fig1,ax=plt.subplots(figsize=(8,6))
        title='Comparing percent of public sequences between cohorts'
        showLegend=True
        nBins=20
        
        ax,ks_p_cohort1_cohort2,t_p_cohort1_cohort2,mean1,mean2,filename=plotHistComprison(data1,data2,data1Name,data2Name,folderToSave,ax,title,showLegend,nBins)
        fig1.savefig(filename,dpi=200)
        plt.show()
       
        #merge tables and get statistics : how much are public in one cohort but not the other:
        mergedPublicList=pd.merge(uniqueSeqDict['PNP'],uniqueSeqDict['Cardio'],how='outer',left_index=True, right_index=True)
        for col in mergedPublicList.columns:
            newCol=col.replace('_x','PNP')
            newCol=newCol.replace('_y','Cardio')
            mergedPublicList= mergedPublicList.rename(columns={col:newCol})
#         print mergedPublicList.head(20)
        
        publicInBoth=mergedPublicList[(mergedPublicList['nSharedPNP'].notnull())&(mergedPublicList['nSharedCardio'].notnull())]
        publicOnlyInPNP=mergedPublicList[(mergedPublicList['nSharedPNP'].notnull())&(mergedPublicList['nSharedCardio'].isnull())]
        publicOnlyInCardio=mergedPublicList[(mergedPublicList['nSharedPNP'].isnull())&(mergedPublicList['nSharedCardio'].notnull())]
        
        print 'number of sequence shared by %s or more samples in both cohorts: %s (%s perc)' %(nSharedMin,
                                                    len(publicInBoth),100*float( len(publicInBoth))/len(mergedPublicList))
        print 'number of sequence shared by %s or more samples only in the PNP cohort: %s (%s perc)' %(nSharedMin,
                                                            len(publicOnlyInPNP),100*float(len(publicOnlyInPNP))/len(mergedPublicList))
        print 'number of sequence shared by %s or more samples only in the Cardio cohort: %s (%s perc)' %(nSharedMin,
                                                            len(publicOnlyInCardio),100*float( len(publicOnlyInCardio))/len(mergedPublicList))
        print 'total=%s' %(len(publicInBoth)+len(publicOnlyInPNP)+len(publicOnlyInCardio))
        print 'merged public list length length=%s' %len(mergedPublicList)
        
        #plot venn diagram:
        from matplotlib_venn import venn2
        fig3,ax3=plt.subplots(figsize=(8,6))
        venn=venn2([set(uniqueSeqDict['PNP'].index.tolist()), set(uniqueSeqDict['Cardio'].index.tolist())],
                   set_labels = ('PNP', 'Cardio'),ax=ax3)
        ax3.set_title('Number of Public sequences in each cohort')
        VennPlotFile='%s/PublicSeq_VennPlot_nSharedMin%s' %(publicAnalysisFolder,nSharedMin)
        fig3.savefig(VennPlotFile,dpi=200)
        plt.show()
        
        #add sequence identities:
        print 'analyzing public sequence identities...'
        identsFile='%s/TCR CDR3 sequence databases/CDR3identityTable_06082018.xlsx' %MyPath
        identsProcessedFile='%s/TCR CDR3 sequence databases/CDR3identityTable_06082018_processed.xlsx' %MyPath
        identsDF=pd.read_excel(identsFile)
        identsProcessedDF=pd.read_excel(identsProcessedFile)
        
        publicDFlist=[('Both',publicInBoth),('onlyPNP',publicOnlyInPNP),('onlyCardio',publicOnlyInCardio)]
        fig4,axes=plt.subplots(nrows=1,ncols=3,figsize=(30,8))
        valueCountDict={}
        for n,publicDF in enumerate(publicDFlist):
            print n,publicDF[0]
            publicDFwithIdents=pd.merge(publicDF[1],identsDF,how='left',left_index=True,right_index=True)
            f1='%s/publicDFwithIdents_%s.xlsx' %(publicAnalysisFolder,publicDF[0])
            publicDFwithIdents.to_excel(f1)
            
            publicDFwithIdents_processed=pd.merge(publicDF[1],identsProcessedDF,how='left',left_index=True,right_index=True)
            df=publicDFwithIdents_processed
            ax=axes[n]
            nSharedThreshold=None
            useMore=True
            column=identityColumnForPie
            dropna=True          
            size=None
            ax,valueCountsgrouped2=plot_identity_pie_plot(ax,df,nSharedThreshold,useMore,column,dropna,size)
            valueCountDict[publicDF[0]]=valueCountsgrouped2
#             print valueCountsgrouped2
        publicIdentsFigFile='%s/publicIdentsFig_%s' %(publicAnalysisFolder,identityColumnForPie)
                
        #generate contingency table and calculate chi test:
        valueCountComb=pd.merge(pd.DataFrame(valueCountDict['onlyPNP']),pd.DataFrame(valueCountDict['onlyCardio']),how='outer',
                               left_index=True,right_index=True)
        valueCountComb2=valueCountComb.T.fillna(0)
#         print valueCountComb2
        from scipy.stats import chi2_contingency
        chi, p, dof, expctd = chi2_contingency(valueCountComb2)
        ax.annotate('p_chi square test (only PNP vs. only Cardio=%s' %round(p,4), xy=(0.96, 1.15), xycoords='axes fraction', 
                    fontsize=22, horizontalalignment='right', verticalalignment='top', fontweight='bold')
        fig4.subplots_adjust(left=0.09, right=0.9, top=0.8, bottom=0.02, wspace=0.5,hspace=0.1)
        fig4.savefig(publicIdentsFigFile,dpi=200)
        plt.show()
        
        print 'end of function!'
    
    return FeatureMeanSummary_all_dataSets
    
    

    

In [ ]:
def plot_identity_pie_plot(ax,df,nSharedThreshold,useMore,column,dropna,size=None):
    
    #(1) DEFINE PARAMETERS:
    if nSharedThreshold is None:
        df2=df.copy()
        title='TCR identites distribution (%s),(%s sequences)' %(column,len(df2))
    else:
        if useMore:
            df2=df[df['nShared']>=nSharedThreshold]
            sharedBy='more'
        else:
            df2=df[df['nShared']<=nSharedThreshold]
            sharedBy='less'
        title='TCR identites distribution (%s)\nSequences shared by %s or %s samples\n(%s sequences)' %(column,nSharedThreshold,sharedBy,len(df2))
    
    #(2) CALCULATE NORMALIZED NUMBER OF IDENTITITES AND PLOT
    valueCounts=pd.DataFrame(df2[column].value_counts(dropna=dropna,normalize=True))
    valueCounts['count2']=np.where(valueCounts[column]>valueCounts[column].min()*2,
                                             valueCounts.index,'Other')
    valueCounts['count2']=valueCounts['count2'].astype(str)
    valueCountsgrouped=valueCounts.groupby('count2').sum()
    #print valueCountsgrouped.head()
    valueCountsgrouped=valueCountsgrouped.sort_values(by=column,ascending=False)
    cmap = plt.cm.get_cmap('Spectral')
    colors =[cmap(x*20) for x in range(len(valueCountsgrouped))]

    ax.pie(valueCountsgrouped[column],labels=valueCountsgrouped.index,autopct='%.2f',colors=colors,textprops={'fontsize': 14})
#     valueCountsgrouped[column].plot.pie(ax=ax,autopct='%.2f')
    ax.set_ylabel('')
    ax.set_title(title,fontsize=20)
    
    #(3) CALCULATE NON-NORMALIZED NUMBER OF IDENTITES TO ENABLE RETURNING COUNTS:
    valueCounts2=pd.DataFrame(df2[column].value_counts(dropna=dropna,normalize=False))
    valueCounts2['count2']=np.where(valueCounts2[column]>valueCounts2[column].min()*2,
                                             valueCounts2.index,'Other')
    valueCounts2['count2']=valueCounts2['count2'].astype(str)
    valueCountsgrouped2=valueCounts2.groupby('count2').sum()

    
    return ax,valueCountsgrouped2

## run function

In [ ]:
ss=12500
repeat=1

datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss%srep%s' %(MyPath,ss,repeat)

PNPListFile='%s/PNP530ss12500_samples_matchedTo_Cardio126ss12500_n70_resByGenderMaleAgeSmokingPastSmokingYes_rs5' %datasetFolder
with open(PNPListFile,'rb') as fp:
    PNPmatchedSampleList=pickle.load(fp)

CardioListFile='%s/Cardio126ss12500_samples_matchedTo_PNP530ss12500_n70_resByGenderMaleAgeSmokingPastSmokingYes_rs5' %datasetFolder
with open(CardioListFile,'rb') as fp:
    CardioMatchedSampleList=pickle.load(fp)
    
delFolderFirst=True
copySamples=True

compareFeatures=True

genOrigComTCRdf=True
genComTCRdf=True
n_comp=10
percSharedCom=10

compareSharing=True
genSepTCRdf=True
nSharedMin=2
identityColumnForPie='Pathology_McPAS' #'Epitope species_VDJDB'/'Pathology_McPAS'

FeatureMeanSummary_all_dataSets=compareTCRforMatchedCohorts(ss,repeat,PNPmatchedSampleList,CardioMatchedSampleList,
                            copySamples,compareFeatures,compareSharing, genSepTCRdf,nSharedMin,delFolderFirst,
                            genOrigComTCRdf,genComTCRdf,n_comp,percSharedCom,identityColumnForPie)




In [ ]:
identsFile='%s/TCR CDR3 sequence databases/CDR3identityTable_06082018.xlsx' %MyPath
identsProcessedFile='%s/TCR CDR3 sequence databases/CDR3identityTable_06082018_processed.xlsx'  %MyPath
identsDF=pd.read_excel(identsFile)
identsProcessedDF=pd.read_excel(identsProcessedFile)

print  identsProcessedDF.index
identsProcessedDF.head()

In [ ]:
a=set([2,3,4,5,6,7])
b=set([1,2])

fig3,ax3=plt.subplots(figsize=(8,6))

from matplotlib_venn import venn2
venn=venn2([a, b],ax=ax3)
f1='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss12500rep1/publicAnalysis/trial2' %MyPath
fig3.savefig(f1, dpi=200)
plt.show()


In [ ]:
from scipy.stats import chi2_contingency